In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet 

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim

from google.cloud import bigquery

[nltk_data] Downloading package wordnet to /home/traffic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/traffic/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/traffic/.local/lib/python3.8/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [2]:
# authenticate with BigQuery API
client = bigquery.Client.from_service_account_json('../../datascience-abovezero-58d98dcf7f65.json')
# Perform a query.
QUERY = ('SELECT * FROM `datascience-abovezero.ml_sandbox.chegg_influencers_comments`')
query_job = client.query(QUERY)  # API request
df =  query_job.result().to_dataframe() #transform to Pandas Dataframe

#df2 = pd.read_csv('analisis_comments_tiktok.csv')# Parameters tuning using Grid Search

# Bag Of Words

### Preprocessing 

In [3]:
#def spelling_correcter(text):
    

def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

comments = df.comment.tolist()
comments_prep = [preprocess(str(comment)) for comment in comments]
comments_dict = gensim.corpora.Dictionary(comments_prep)
bow_corpus = [comments_dict.doc2bow(doc) for doc in comments_prep]

In [4]:
'''
Checking dictionary created
'''
count = 0
for k, v in comments_dict.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 ayyyi
1 legend
2 call
3 colleg
4 univers
5 amaz
6 person
7 antedragz
8 chegg
9 fuck
10 gift


### Training the model

### Testing the model

# TFIDF 

### Preprocessing

In [5]:
#docs_raw = df.comment.fillna('').tolist()

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 0.01)
#dtm_tf = tf_vectorizer.fit_transform(docs_raw)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
#dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)

### Training The Model 

In [7]:
def get_top_topics(model, vectorizer, topn=5):
    names = tf_vectorizer.get_feature_names()
    res = {}
    for i_cluster, cluster in enumerate(model.components_):
        res[i_cluster]=[]
        for i_feature in cluster.argsort()[:-topn - 1:-1]:
            res[i_cluster] = res[i_cluster] + [names[i_feature]]
    return res

In [8]:
#first get topics for all comments (Full Dataset)
docs_raw_full = df.comment.fillna('').tolist()
dtm_tf_full = tf_vectorizer.fit_transform(docs_raw_full)
# train model to find topics per influencer
lda_tf_full = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tf_full.fit(dtm_tf_full)
# extract most important topics
topics_full =  get_top_topics(lda_tf_full, tf_vectorizer, topn=5)
print(topics_full)

X_test = tf_vectorizer.transform(df.loc[:,'comment'])

doc_topic_dist_unnormalized_full = np.matrix(lda_tf_full.transform(X_test))

/home/traffic/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{0: ['just', 'people', 'good', 'college', 'chegg'], 1: ['cancer', 'school', 'colon', 'high', 'year'], 2: ['love', 'video', 'thank', 'simone', 'youtube'], 3: ['quot', 'don', 'time', 'like', 'cancer']}


In [50]:
res_ = pd.DataFrame(doc_topic_dist_unnormalized_full.argmax(axis=1)).value_counts().rename_axis('topic').reset_index(name='counts')
counts_sum = res_.counts.sum()
res_.loc[:,'counts'] = res_.counts.apply(lambda x:round(x/counts_sum*100,1))
res_

,topic,counts
0,0,48.9
1,2,18.7
2,3,16.3
3,1,16.1


In [43]:
pd.DataFrame(doc_topic_dist_unnormalized_full.argmax(axis=1)).value_counts().rename_axis('topic').reset_index(name='counts')

,topic,counts
0,0,1676
1,2,642
2,3,558
3,1,551


In [75]:
#first get topics for all comments (Full Dataset)
docs_raw_full = df.comment.fillna('').tolist()
dtm_tf_full = tf_vectorizer.fit_transform(docs_raw_full)
# train model to find topics per influencer
lda_tf_full = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tf_full.fit(dtm_tf_full)
# extract most important topics
topics_full =  get_top_topics(lda_tf_full, tf_vectorizer, topn=5)
print(topics_full)

#now let's what of those topics are talked about by what influencer
# and then go more granular and look at topics on influencer level.
topic_mapping = pd.DataFrame()
res_infl = pd.DataFrame({'influencer':[]})
for topic in topics_full:
    topic_mapping.loc[:,topic] = np.array([str(topics_full[topic])])
    res_infl[topic] = []

for i_infl, infl in enumerate(df.influencer.unique()):
    dfi = df.loc[df.influencer==infl,:]
    #what categories of full topic model do infl comments belong to
    X_test = tf_vectorizer.transform(dfi.loc[:,'comment'])
    doc_topic_dist_unnormalized_full = np.matrix(lda_tf_full.transform(X_test))
    # get count of number topics are 'hit'
    res_ = pd.DataFrame({'topic':list(topics_full.keys())})
    res__ = pd.DataFrame(doc_topic_dist_unnormalized_full.argmax(axis=1)).value_counts().rename_axis('topic').reset_index(name='counts')
    res_ = res_.merge(res__, on='topic', how='left').reset_index(drop=True).fillna(0)
    display(res_)
    # calculate relative topic distribution for influencer
    counts_sum = res_.counts.sum()
    res_.loc[:,'counts'] = res_.counts.apply(lambda x:round(x/counts_sum*100,1))
    res_ = res_.sort_values('topic',ascending=True).reset_index(drop=True)
    display([infl] + list(res_.counts))
    res_infl.loc[len(res_infl)] = [infl] + list(res_.counts)
    #for comment, comment_orig in zip(doc_topic_dist_unnormalized_full, dfi.comment):
    #    print(comment_orig)
    #    print(topics[comment[0].argmax()])
        
        
    """  
    docs_raw = dfi.comment.fillna('').tolist()
    dtm_tf = tf_vectorizer.fit_transform(docs_raw)
    # train model to find topics per influencer
    lda_tf = LatentDirichletAllocation(n_components=4, random_state=0)
    lda_tf.fit(dtm_tf)
    # extract most important topics
    topics =  get_top_topics(lda_tf, tf_vectorizer, topn=5)
    print(infl)
    print(topics)
    
    doc_topic_dist_unnormalized = np.matrix(lda_tf.transform(X_test))
    for comment, comment_orig in zip(doc_topic_dist_unnormalized, dfi.comment):
        print(comment_orig)
        print(topics[comment[0].argmax()])
    """
    #if i_infl>-1:
    #    break

{0: ['just', 'people', 'good', 'college', 'chegg'], 1: ['cancer', 'school', 'colon', 'high', 'year'], 2: ['love', 'video', 'thank', 'simone', 'youtube'], 3: ['quot', 'don', 'time', 'like', 'cancer']}


/home/traffic/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,topic,counts
0,0,38
1,1,5
2,2,4
3,3,2


['itssozer', 77.6, 10.2, 8.2, 4.1]

,topic,counts
0,0,15.0
1,1,2.0
2,2,2.0
3,3,0.0


...

['campuscrawl', 77.4, 2.2, 10.8, 9.7]

,topic,counts
0,0,29
1,1,4
2,2,3
3,3,4


['mathicallytutors', 72.5, 10.0, 7.5, 10.0]

,topic,counts
0,0,13.0
1,1,0.0
2,2,2.0
3,3,0.0


['leonthetutor', 86.7, 0.0, 13.3, 0.0]

In [74]:
res_infl

,influencer,0,1,2,3
0,itssozer,77.6,10.2,8.2,4.1
1,emilyballz,78.9,10.5,10.5,0.0
2,iamalilstitious,60.0,0.0,20.0,20.0
3,sarahbada_,78.9,10.5,5.3,5.3
4,sakshammagic,80.8,0.0,7.7,11.5
5,cloutom,72.0,10.7,8.0,9.3
6,medstudebt,60.0,0.0,0.0,40.0
7,j0siahyoung,90.0,0.0,10.0,0.0
8,modern.day.classic,64.6,6.2,24.6,4.6
9,sarahrav,76.2,4.8,9.5,9.5
